In [1]:
##Load the necessary Libraries
!pip install farasapy
import re
import string
from farasa.stemmer import FarasaStemmer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk as nltk
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
#Filtering out the warnings

import warnings

warnings.filterwarnings('ignore')

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
##Unicode
HAMZA = u'\u0621'
ALEF_MADDA = u'\u0622'
ALEF_HAMZA_ABOVE = u'\u0623'
WAW_HAMZA = u'\u0624'
ALEF_HAMZA_BELOW = u'\u0625'
YEH_HAMZA = u'\u0626'
ALEF = u'\u0627'
BEH = u'\u0628'
TEH_MARBUTA = u'\u0629'
TEH = u'\u062a'
THEH = u'\u062b'
JEEM = u'\u062c'
HAH = u'\u062d'
KHAH = u'\u062e'
DAL = u'\u062f'
THAL = u'\u0630'
REH = u'\u0631'
ZAIN = u'\u0632'
SEEN = u'\u0633'
SHEEN = u'\u0634'
SAD = u'\u0635'
DAD = u'\u0636'
TAH = u'\u0637'
ZAH = u'\u0638'
AIN = u'\u0639'
GHAIN = u'\u063a'
TATWEEL = u'\u0640'
FEH = u'\u0641'
QAF = u'\u0642'
KAF = u'\u0643'
LAM = u'\u0644'
MEEM = u'\u0645'
NOON = u'\u0646'
HEH = u'\u0647'
WAW = u'\u0648'
ALEF_MAKSURA = u'\u0649'
YEH = u'\u064a'
MADDA_ABOVE = u'\u0653'
HAMZA_ABOVE = u'\u0654'
HAMZA_BELOW = u'\u0655'
ZERO = u'\u0660'
ONE = u'\u0661'
TWO = u'\u0662'
THREE = u'\u0663'
FOUR = u'\u0664'
FIVE = u'\u0665'
SIX = u'\u0666'
SEVEN = u'\u0667'
EIGHT = u'\u0668'
NINE = u'\u0669'
PERCENT = u'\u066a'
DECIMAL = u'\u066b'
THOUSANDS = u'\u066c'
STAR = u'\u066d'
MINI_ALEF = u'\u0670'
ALEF_WASLA = u'\u0671'
FULL_STOP = u'\u06d4'
BYTE_ORDER_MARK = u'\ufeff'

In [3]:
df = pd.read_csv('arabic_news2.csv')
df = df[['comman', 'text', 'label']]
df = df[['label', 'text']]
df.head()

,label,text
0,0,مفتشو الأسلحة الكيمياوية يباشرون دورهم في دوما...
1,0,الدولار قرب أدنى مستوى والدولار يتدهور
2,0,تم تحرير جنوبي سوريا من يد تنظيم الدولة
3,0,هدوء بين الجيش الإسرائيلي ومتظاهرين فلسطينيين ...
4,0,المرأة مضطهدة في عام 2018


In [4]:
#1
emoji_group =u"\U0001F600-\U0001F64F", u"\U0001F300-\U0001F5FF", u"\U0001F680-\U0001F6FF", u"\U0001F1E0-\U0001F1FF"  
def remove_emojis(text):
    emoji_pattern = re.compile("emoji_group", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
#2
def remove_non_arabic(text):
    return ' '.join(re.sub(u"[^\u0621-\u063A\u0640-\u0652 ]", " ", text,  flags=re.UNICODE).split())
#3
def strip_tatweel(text):
    return re.sub(u'[%s]' % TATWEEL, '', text)
#4
def remove_underscore(text):
    return ' '.join(text.split('_'))
#5
def remove_extra_spaces(text):
    return ' '.join(text.split())
#6
def remove_retweet_tag(text):
    return re.compile('\#').sub('', re.compile('rt @[a-zA-Z0-9_]+:|@[a-zA-Z0-9_]+').sub('', text).strip())
df['C_non_ar'] = df['text'].apply(lambda x: remove_non_arabic(x))
df['C_tatweel'] = df['C_non_ar'].apply(lambda x: strip_tatweel(x))
df['C__underscore'] = df['C_tatweel'].apply(lambda x: remove_underscore(x))
df['C_extra_spaces'] = df['C__underscore'].apply(lambda x: remove_extra_spaces(x))
df['C_retweet_tag'] = df['C_extra_spaces'].apply(lambda x: remove_retweet_tag(x))
df['C_emojis'] = df['C_retweet_tag'].apply(lambda x: remove_emojis(x))

df = df[['label', 'text', 'C_emojis']]
df = df.rename(columns={'text': 'old_text', 'C_emojis': 'text'})
df = df[['label', 'text']]
df.head()
  

,label,text
0,0,مفتشو الأسلحة الكيمياوية يباشرون دورهم في دوما...
1,0,الدولار قرب أدنى مستوى والدولار يتدهور
2,0,تم تحرير جنوبي سوريا من يد تنظيم الدولة
3,0,هدوء بين الجيش الإسرائيلي ومتظاهرين فلسطينيين ...
4,0,المرأة مضطهدة في عام


In [5]:
#1
#punctuation
import string
string.punctuation
def remove_punct(text):
    
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

df['nopunc'] = df['text'].apply(lambda x: remove_punct(x))

#2
#tokenization
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
def tokenize(text):
    tokens = word_tokenize(text)
    return tokens
df['tokenized'] = df['nopunc'].apply(lambda x: tokenize(x))

#3
#stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_Ar = nltk.corpus.stopwords.words('arabic')
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopwords_Ar]
    return text

df['nostop'] = df['tokenized'].apply(lambda x: remove_stopwords(x))

#4
#stemmer
!pip install snowballstemmer
from snowballstemmer import stemmer
ar_stem = stemmer("arabic")
import nltk
ps = nltk.PorterStemmer()
def stemming(tokenized_text):
    text = [ps.stem(word) for word in tokenized_text]
    return text

df['stemmed'] = df['nostop'].apply(lambda x: stemming(x))

### Create function to remove punctuation, tokenize, remove stopwords, and stem

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
     #tokens = re.split('\W+', text)
    tokens = word_tokenize(text)
    text = " ".join([ps.stem(word) for word in tokens if word not in stopwords_Ar])
    return text
df=df[['label','text']]
df['cleaned_text'] = df['text'].apply(lambda x: clean_text(x))

from sklearn.feature_extraction.text import CountVectorizer

# To create a Count Vectorizer, we simply need to instantiate one.
# There are special parameters we can set here when making the vectorizer, but
# for the most basic example, it is not needed.


vectorizer = CountVectorizer()
features_CountVec = vectorizer.fit_transform(df['cleaned_text'])
print(features_CountVec.shape)
print('Sparse Matrix :\n', features_CountVec)
#features_CountVec = pd.DataFrame(features_CountVec.toarray())
features_CountVec = pd.DataFrame(features_CountVec)

features_CountVec

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


(3788, 6401)
Sparse Matrix :
   (0, 5477)	1
  (0, 666)	1
  (0, 1697)	1
  (0, 6069)	1
  (0, 3603)	1
  (0, 3613)	1
  (0, 1348)	1
  (1, 1208)	1
  (1, 4468)	1
  (1, 71)	1
  (1, 5364)	1
  (1, 5850)	1
  (1, 6096)	1
  (2, 3173)	1
  (2, 2832)	1
  (2, 3355)	1
  (2, 3872)	1
  (2, 6170)	1
  (2, 3213)	1
  (2, 1210)	1
  (3, 5769)	1
  (3, 1090)	1
  (3, 754)	1
  (3, 6024)	1
  (3, 4386)	1
  :	:
  (3784, 5100)	1
  (3784, 1375)	1
  (3784, 3795)	1
  (3784, 5163)	1
  (3784, 4334)	1
  (3785, 1318)	1
  (3785, 704)	1
  (3785, 1739)	1
  (3785, 2072)	1
  (3785, 940)	1
  (3785, 3135)	1
  (3785, 6386)	1
  (3785, 3144)	1
  (3786, 1362)	1
  (3786, 2904)	1
  (3786, 3682)	1
  (3786, 1448)	1
  (3786, 1062)	1
  (3787, 1999)	1
  (3787, 5759)	1
  (3787, 700)	1
  (3787, 4800)	1
  (3787, 3752)	1
  (3787, 222)	1
  (3787, 1130)	1


,0
0,"(0, 5477)\t1\n (0, 666)\t1\n (0, 1697)\t1\..."
1,"(0, 1208)\t1\n (0, 4468)\t1\n (0, 71)\t1\n..."
2,"(0, 3173)\t1\n (0, 2832)\t1\n (0, 3355)\t1..."
3,"(0, 5769)\t1\n (0, 1090)\t1\n (0, 754)\t1\..."
4,"(0, 1807)\t1\n (0, 5427)\t1\n (0, 4162)\t1"
...,...
3783,"(0, 1716)\t1\n (0, 1483)\t1\n (0, 3320)\t1..."
3784,"(0, 5584)\t1\n (0, 5100)\t1\n (0, 1375)\t1..."
3785,"(0, 1318)\t1\n (0, 704)\t1\n (0, 1739)\t1\..."
3786,"(0, 1362)\t1\n (0, 2904)\t1\n (0, 3682)\t1..."


In [10]:
# Set parameters
max_features = 10000
embedding_dim = 100
maxlen = 300
batch_size = 32
epochs = 10
# Split the dataset into training and testing sets
X = df['cleaned_text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Tokenize and pad the text sequences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen)

In [11]:
#CNN

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Sequential

# Define hyperparameters
max_features = 10000
embedding_dim = 100
maxlen = 300
batch_size = 32
epochs = 10

# Load and preprocess data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=max_features)
X_train_pad = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_test_pad = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

# Build the CNN model
model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(Conv1D(256, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])
model.summary()

history = model.fit(X_train_pad, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test_pad, y_test))

# Evaluate the model on the test set
loss, accuracy, auc = model.evaluate(X_test_pad, y_test)
print(f'Model\tAccuracy\tLoss\tAUC')
print(f'CNN\t{accuracy:.2f}\t{loss:.2f}\t{auc:.2f}')

17464789/17464789 [==============================] - 73s 4us/step
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 300, 100)          1000000   
                                                                 
 conv1d_3 (Conv1D)           (None, 296, 256)          128256    
                                                                 
 global_max_pooling1d_3 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_8 (Dense)             (None, 64)                16448     
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_9 (Dense)             (None, 1)                

In [13]:
#LSTM

In [14]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

# Build the LSTM model
model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])
model.summary()

history = model.fit(X_train_pad, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test_pad, y_test))

# Evaluate the model on the test set and print the results
loss, accuracy, auc = model.evaluate(X_test_pad, y_test)
print(f'Model\tAccuracy\tLoss\tAUC')
print(f'LSTM\t{accuracy:.2f}\t{loss:.2f}\t{auc:.2f}')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 300, 100)          1000000   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,125,569
Trainable params: 1,125,569
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


UnimplementedError: Graph execution error:

Detected at node 'binary_crossentropy/Cast' defined at (most recent call last):
    File "C:\ProgramData\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\ProgramData\Anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\ProgramData\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\ProgramData\Anaconda3\lib\asyncio\base_events.py", line 601, in run_forever
      self._run_once()
  

In [ ]:
#BiLSTM

In [ ]:
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

# Build the BiLSTM model
model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])
model.summary()

history = model.fit(X_train_pad, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test_pad, y_test))

# Evaluate the model on the test set
loss, accuracy, auc = model.evaluate(X_test_pad, y_test)
print(f'Model\tAccuracy\tLoss\tAUC')
print(f'BiLSTM\t{accuracy:.6f}\t{loss:.6f}\t{auc:.6f}')

In [ ]:
#CNN+LSTM

In [ ]:
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, concatenate
from tensorflow.keras.models import Sequential

# Build the CNN+LSTM model
model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])
model.summary()

history = model.fit(X_train_pad, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test_pad, y_test))

# Evaluate the model on the test set
loss, accuracy, auc = model.evaluate(X_test_pad, y_test)
print(f'Model\tAccuracy\tLoss\tAUC')
print(f'CNN + LSTM\t{accuracy:.6f}\t{loss:.6f}\t{auc:.6f}')

In [ ]:
#CNN+BiLSTM

In [ ]:
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout, concatenate
from tensorflow.keras.models import Sequential

# Build the CNN+BiLSTM model
model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])
model.summary()

history = model.fit(X_train_pad, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test_pad, y_test))

# Evaluate the model on the test set
loss, accuracy, auc = model.evaluate(X_test_pad, y_test)
print(f'Model\tAccuracy\tLoss\tAUC')
print(f'CNN + BiLSTM\t{accuracy:.6f}\t{loss:.6f}\t{auc:.6f}')

In [33]:
# CNN
loss, accuracy, auc = model_cnn.evaluate(X_test_pad, y_test)
print(f'Model\tAccuracy\tLoss\tAUC')
print(f'CNN\t{accuracy:.6f}\t{loss:.6f}\t{auc:.6f}')

# LSTM
loss, accuracy, auc = model_lstm.evaluate(X_test_pad, y_test)
print(f'Model\tAccuracy\tLoss\tAUC')
print(f'LSTM\t{accuracy:.6f}\t{loss:.6f}\t{auc:.6f}')

# CNN + LSTM
loss, accuracy, auc = model_cnn_lstm.evaluate(X_test_pad, y_test)
print(f'Model\tAccuracy\tLoss\tAUC')
print(f'CNN + LSTM\t{accuracy:.6f}\t{loss:.6f}\t{auc:.6f}')

# BiLSTM
loss, accuracy, auc = model_bilstm.evaluate(X_test_pad, y_test)
print(f'Model\tAccuracy\tLoss\tAUC')
print(f'BiLSTM\t{accuracy:.6f}\t{loss:.6f}\t{auc:.6f}')

# CNN + BiLSTM
loss, accuracy, auc = model_cnn_bilstm.evaluate(X_test_pad, y_test)
print(f'Model\tAccuracy\tLoss\tAUC')
print(f'CNN + BiLSTM\t{accuracy:.6f}\t{loss:.6f}\t{auc:.6f}')

Model	        Accuracy	         Loss	           AUC
CNN	        0.872240	     0.738599	     0.928871
LSTM	        0.872240	     0.738599	     0.928871
CNN + LSTM	0.872240	     0.738599	     0.928871
BiLSTM	        0.872240	     0.738599	     0.928871
CNN + BiLSTM	0.872240	     0.738599	     0.928871
